### **Optimización de hiperparámetros**

Este notebook tiene como único propósito la optimización de hiperparámetros para su aplicación en nuestro RandomForest, y posiblemente obtener mejores resultados. Los métodos utilizados son muy exigentes computacionalmente, es por ello que se realiza este proceso en un notebook diferente, para hacerlo una vez.

In [9]:
#librerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_regression, mutual_info_classif
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [10]:
#lectura de datos
data = pd.read_csv(r"Datos/dataset.csv")
labels = pd.read_csv(r"Datos/labels.csv")

#### **Manipulación de datos**
Pues es básicamente una copia de lo que hice en el notebook del proyecto, solo es para tener el mismo dataframe al momento de hacer el GridSearch y RandomSearch.

In [15]:
data_limpio = data.copy(deep=True)
data_limpio.drop('En.Anterior.', inplace=True, axis=1)

var_thr = VarianceThreshold(threshold = 0.01)

var_thr.fit(data_limpio.iloc[:,2: ])
var_thr.get_support()
concol = [column for column in data_limpio.iloc[:,2: ].columns 
          if column not in data_limpio.iloc[:,2: ].columns[var_thr.get_support()]]


data_limpio.drop(concol, inplace=True, axis=1)
data_numeric = pd.get_dummies(data['idEye'])
data_num = pd.concat([data , data_numeric], axis=1).reindex(data.index)
data_num.drop(columns=['idEye', 'En.Anterior.'], axis=1, inplace=True)

le = LabelEncoder()
data_le = data_limpio.copy(deep = True)
label = le.fit_transform(data_le['idEye'].values)
data_le.drop('idEye', axis=1, inplace=True)
data_le['idEye'] = label

X =data_le.iloc[:,2:].values
y= labels.iloc[:,-1].values

for i in range(X.shape[1]):
    X[:,i] = le.fit_transform(X[:,i])
y = le.fit_transform(y)

standard_scaler = StandardScaler()

X = standard_scaler.fit_transform(X)
classes = np.unique(y)
nClasses = len(classes)

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.85,random_state=42)

#### **Hiperparámetros del RandomForest**

<!-- Para la optimización se utilizarán dos métodos distintos, que son muy parecidos. Esto no es tan necesario, pero si ayuda a sacar algo mas de rendimiento o precisión en nuestro modelo. -->

##### - **GridSearch**
Consiste en básicamente buscar combinaciones de parámetros de manera controlada, para así encontrar los valores que mejor se ajusten al modelo que estamos utilizando.
##### - **RandomSearch**
Nuevamente realiza distintas combinaciones de parámetros, ahora de manera aleatoria, a partir de ciertas iteraciones y dentro del rango de los parámetros que le pongamos.

In [13]:
param_grid = {
    'n_estimators': (200, 300, 400, 500, 600, 700),
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=None)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                          cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print("Parámetros:", grid_search.best_params_)
best_model1 = grid_search.best_estimator_

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Parámetros: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}


In [14]:
param_dist = {
    'n_estimators': randint(200, 700),
    'max_depth': randint(0, 110),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=None)
random_search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                 n_iter=100, cv=5, n_jobs=-1, verbose=2)
random_search.fit(X_train, y_train)

print("Parámetros:", random_search.best_params_)
best_model2 = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Parámetros: {'bootstrap': False, 'max_depth': 105, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 7, 'n_estimators': 693}
